# Análise do Risco Geo-Hidrológico e Acidentes

https://www.gov.br/cemaden/pt-br/assuntos/riscos-geo-hidrologicos

In [26]:
# Importing Dependencies
import geopandas as gpd
import fiona
import scipy.stats as stats
from shapely.geometry import Point
import pandas as pd
from datetime import datetime

In [2]:
# Read KML File from January Hidrologic Risk
fiona.drvsupport.supported_drivers["KML"] = "rw"
my_map = gpd.read_file("mapas/Janeiro.kml", driver="KML")
my_map

,Name,Description,geometry
0,Janeiro2,,"POLYGON Z ((-43.62187 -23.07444 0.00000, -42.6..."


In [3]:
# Read DataTran Base from 2022
data = pd.read_csv("raw_data/datatran_2022.csv")
data.head()

,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,...,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop
0,405151,1/1/2022,sábado,1:35:00,PI,316.0,415.0,MARCOLANDIA,Ingestão de álcool pelo condutor,Colisão traseira,...,0,1,0,1,2,-7.432800,-40.682619,SPRF-PI,DEL04-PI,UOP03-DEL04-PI
1,405158,1/1/2022,sábado,2:40:00,PR,116.0,33.0,CAMPINA GRANDE DO SUL,Ingestão de álcool pelo condutor,Tombamento,...,0,0,0,1,1,-25.114403,-48.846755,SPRF-PR,DEL01-PR,UOP02-DEL01-PR
2,405172,1/1/2022,sábado,5:22:00,MS,163.0,393.0,NOVA ALVORADA DO SUL,Condutor deixou de manter distância do veículo...,Colisão traseira,...,0,1,0,1,2,-21.228445,-54.456296,SPRF-MS,DEL02-MS,UOP01-DEL02-MS
3,405203,1/1/2022,sábado,7:00:00,RJ,101.0,457.0,ANGRA DOS REIS,Reação tardia ou ineficiente do condutor,Colisão frontal,...,1,1,0,1,2,-23.031498,-44.177153,SPRF-RJ,DEL03-RJ,UOP02-DEL03-RJ
4,405207,1/1/2022,sábado,9:00:00,MG,40.0,508.3,RIBEIRAO DAS NEVES,Acumulo de água sobre o pavimento,Saída de leito carroçável,...,0,3,0,0,1,-19.760612,-44.134754,SPRF-MG,DEL02-MG,UOP01-DEL02-MG


In [4]:
# Collecting month from the accident
def get_month(row: pd.Series):
    # Concatenando data e hora do acidente
    date: datetime = datetime.strptime(row["data_inversa"], "%m/%d/%Y")
    return date.month


data["month"] = data.apply(get_month, axis=1)

In [5]:
# Capturing Geo information from accidents in January
jan_accidents = data.loc[data["month"] == 1][
    ["uf", "longitude", "latitude", "causa_acidente"]
]
jan_accidents

,uf,longitude,latitude,causa_acidente
0,PI,-40.682619,-7.432800,Ingestão de álcool pelo condutor
1,PR,-48.846755,-25.114403,Ingestão de álcool pelo condutor
2,MS,-54.456296,-21.228445,Condutor deixou de manter distância do veículo...
3,RJ,-44.177153,-23.031498,Reação tardia ou ineficiente do condutor
4,MG,-44.134754,-19.760612,Acumulo de água sobre o pavimento
...,...,...,...,...
61541,AL,-35.842455,-8.950803,Animais na Pista
61957,MG,-44.966878,-17.381781,Velocidade Incompatível
64240,PE,-34.931767,-7.983443,Acesso irregular
64444,MG,-43.210153,-19.849754,Pista esburacada


In [16]:
# Registring accidents location as Geometric Points
geometic_points = []

for xy in zip(jan_accidents["longitude"], jan_accidents["latitude"]):
    geometic_points.append(Point(xy))

accident_locations = gpd.GeoDataFrame(jan_accidents, geometry=geometic_points)
accident_locations.crs = "epsg:4326"
accident_locations['in_my_map'] = False

accident_locations

,uf,longitude,latitude,causa_acidente,geometry,in_my_map
0,PI,-40.682619,-7.432800,Ingestão de álcool pelo condutor,POINT (-40.68262 -7.43280),False
1,PR,-48.846755,-25.114403,Ingestão de álcool pelo condutor,POINT (-48.84676 -25.11440),False
2,MS,-54.456296,-21.228445,Condutor deixou de manter distância do veículo...,POINT (-54.45630 -21.22845),False
3,RJ,-44.177153,-23.031498,Reação tardia ou ineficiente do condutor,POINT (-44.17715 -23.03150),False
4,MG,-44.134754,-19.760612,Acumulo de água sobre o pavimento,POINT (-44.13475 -19.76061),False
...,...,...,...,...,...,...
61541,AL,-35.842455,-8.950803,Animais na Pista,POINT (-35.84246 -8.95080),False
61957,MG,-44.966878,-17.381781,Velocidade Incompatível,POINT (-44.96688 -17.38178),False
64240,PE,-34.931767,-7.983443,Acesso irregular,POINT (-34.93177 -7.98344),False
64444,MG,-43.210153,-19.849754,Pista esburacada,POINT (-43.21015 -19.84975),False


In [18]:
# Checking if accident is inside Risk Zone
for i in accident_locations.index:
    point = accident_locations.geometry.loc[i]
    accident_locations.loc[i, "in_my_map"] = my_map.contains(point).values

accident_locations.head()

,uf,longitude,latitude,causa_acidente,geometry,in_my_map
0,PI,-40.682619,-7.432800,Ingestão de álcool pelo condutor,POINT (-40.68262 -7.43280),True
1,PR,-48.846755,-25.114403,Ingestão de álcool pelo condutor,POINT (-48.84676 -25.11440),False
2,MS,-54.456296,-21.228445,Condutor deixou de manter distância do veículo...,POINT (-54.45630 -21.22845),False
3,RJ,-44.177153,-23.031498,Reação tardia ou ineficiente do condutor,POINT (-44.17715 -23.03150),True
4,MG,-44.134754,-19.760612,Acumulo de água sobre o pavimento,POINT (-44.13475 -19.76061),True


In [19]:
# Collecting accidents directly caused by rain
rain_accidents = pd.concat(
    [
        accident_locations[accident_locations.causa_acidente == "Chuva"],
        accident_locations[
            accident_locations.causa_acidente == "Acumulo de água sobre o pavimento"
        ],
    ]
)
rain_accidents.head()

,uf,longitude,latitude,causa_acidente,geometry,in_my_map
6,MG,-44.361079,-19.333821,Chuva,POINT (-44.36108 -19.33382),True
29,DF,-48.172115,-15.943619,Chuva,POINT (-48.17211 -15.94362),True
72,MG,-42.476167,-19.388008,Chuva,POINT (-42.47617 -19.38801),True
175,ES,-40.630009,-20.380268,Chuva,POINT (-40.63001 -20.38027),True
183,GO,-49.086400,-16.478900,Chuva,POINT (-49.08640 -16.47890),True


In [25]:
# Checking accidents relation with the risk region
total = len(rain_accidents)
accidents_from_risk = sum(rain_accidents.in_my_map)
print(f"Total de Acidentes em Janeiro Causados pela Chuva: {total}")
print(f"Acidentes na região de Risco: {accidents_from_risk}")
print(f"Acidentes fora da região de Risco: {total - accidents_from_risk}")

Total de Acidentes em Janeiro Causados pela Chuva: 246
Acidentes na região de Risco: 112
Acidentes fora da região de Risco: 134
